In [3]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.pardir))


import safetensors
import torch
import torch.nn.functional as F
from accelerate import notebook_launcher
from einops import rearrange
from einops.layers.torch import Rearrange
from simple_parsing import ArgumentParser

from models import brainformer
from utils.data_utils import BrainDataset
from utils.train_utils import TrainConfig, run_train_model, count_parameters

In [4]:
train_config = TrainConfig(exp_name='brainformer_simple', p_augs=0)
data_path = Path(r"D:\Work\brain-to-text-competition\data\competitionData")

train_dataset = BrainDataset(data_path / 'train')
test_dataset = BrainDataset(data_path / 'test')

# Init model
mae_config = brainformer.MAEConfig(window_size=1024)
model = brainformer.MAE(mae_config)
count_parameters(model)

args = (model, (train_dataset, test_dataset), train_config)
notebook_launcher(run_train_model, args, num_processes=1)